In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread,imshow
import sqlite3
from os import listdir
from pathlib import Path
import cv2
from scipy.stats import gaussian_kde

In [3]:
schemaMat = np.array([[366,641],[1463,641],[366,3018],[1463,3018]])
schema = imread('schema.png')
schemaHalf = cv2.imread('schema.png')
schemaHalf =schemaHalf[1829:3658,0:1829]

# Functions for creating new xy coordinates using homography matrix

In [9]:
metaMaster = pd.read_csv('metaMaster.csv')
schemaMat = np.array([[366,641],[1463,641],[366,3018],[1463,3018]])


In [3]:
def homography(dataset, schmat):
    #isolate the court corner coordinates
    footMats = dataset.iloc[:,4:12]
    #create a numpy array of (4,2) using the corner coordinates for each rally
    footMats['mat'] = footMats.apply(lambda row: np.array([row['TLx'],row['TLy'],row['TRx'],row['TRy'],row['BLx'],row['BLy'],row['BRx'],row['BRy']]).reshape(4,2),axis=1)
    footMats = footMats.drop(footMats.columns[0:8],axis=1)
    #using cv2.findHomography, save the homography matrix for each rally to translate onto tennis court schema
    footMats['homog'] = footMats.apply(lambda row: cv2.findHomography(row['mat'],schmat)[0],axis=1)
    
    return footMats['homog']

In [4]:
def inputXY(row, xcol, ycol):
    xy = row[[xcol,ycol]]
    scalarxy = np.array([xy[0],xy[1],1]).reshape(3,1)
    
    return scalarxy


#Inputs, row from bbplayer & row from homog

#Outputs x & y translated player coordinates

def tXYdb(plaXY, homRow, xtoT, ytoT):
    playerScalar = inputXY(plaXY, xtoT, ytoT)
    
    transXY = np.dot(homRow, playerScalar)
    
    transXY = transXY/transXY[2]
    
    return transXY[0:2]





def appendNewXYDB(playerData, homDat, xlab, ylab, newx = 'fX', newy= 'fY'):
    playerData[newx] = playerData.apply(lambda row: tXYdb(row, homDat, xlab, ylab)[0][0], axis =1)
    playerData[newy] = playerData.apply(lambda row: tXYdb(row, homDat, xlab, ylab)[1][0], axis =1)
    
    return playerData

# Function to extract dataframe from .pitg files

In [5]:
def extractBB(filename, homograph , matchNo):
    #Create connection with file
    try:
        conn = sqlite3.connect(filename)    
    except Error as e:
        print(e)
    
    #Pull table names out of db 
    tableNames = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    
    #Create a dictionary with table names as key and data as values
    tableDic = {}

    for i in tableNames.values:
        tableDic[str(i[0])] = pd.read_sql_query('SELECT * FROM ' + i[0], conn)
    
    #close connection
    conn.close()
    
    #Create variables with key-value pairs of interest
    bbplayer = tableDic['bounding_boxes']
    
    #Identify x,y coord for player position and create new rows with this
    
    bbplayer['posX'] = bbplayer.apply(lambda row: (row['x']+(row['width']/2)), axis =1)
    bbplayer['posY'] = bbplayer.apply(lambda row: (row['y']+ row['height']), axis =1)

    newPlay = appendNewXYDB(bbplayer, homograph, 'posX', 'posY', 'fX','fY')
    newPlay['matchID'] = (matchNo + 1)
    newPlay = newPlay.drop(['x', 'y', 'width', 'height', 'mask',
       'posX', 'posY'],axis = 1)
    
    return newPlay
    
  

The method below is specifically for use by the example db provided by Jordan. Not applicable to the main dataset.

In [6]:
def inputXY(row, xcol, ycol):
    xy = row[[xcol,ycol]]
    scalarxy = np.array([xy[0],xy[1],1]).reshape(3,1)
    
    return scalarxy

def homXY(row):
    homarr = row[1:].values
    hommat = homarr.reshape((3,3))
    
    return hommat

# function to identify homography row by frame_id

# INPUTS: row of bbplayer, homography dataset

def getHom(plaRow, homData):
    frame = plaRow['frame_id']
    homogRow = homData.loc[homData['frame_id']==frame,:]
    
    return homogRow.squeeze()
    

def tXY(plaXY, homRow, xtoT, ytoT):
    playerScalar = inputXY(plaXY, xtoT, ytoT)
    homographyMatrix = homXY(homRow)
    
    transXY = np.dot(homographyMatrix, playerScalar)
    
    transXY = transXY/transXY[2]
    
    return transXY[0:2]

def total(playerRow, homographyData, xcoln, ycoln):
    player = playerRow
    homR = getHom(player, homographyData)
    transXY = tXY(player, homR, xcoln, ycoln)
    
    return transXY

def appendNewXY(playerData, homDat, xlab, ylab, newx, newy):
    playerData[newx] = playerData.apply(lambda row: total(row, homDat, xlab, ylab)[0][0], axis =1)
    playerData[newy] = playerData.apply(lambda row: total(row, homDat, xlab, ylab)[1][0], axis =1)
    
    return playerData

def extractDF(filename):
    #Create connection with file
    try:
        conn = sqlite3.connect(filename)    
    except Error as e:
        print(e)
    
    #Pull table names out of db 
    tableNames = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    
    #Create a dictionary with table names as key and data as values
    tableDic = {}

    for i in tableNames.values:
        tableDic[str(i[0])] = pd.read_sql_query('SELECT * FROM ' + i[0], conn)
    
    #close connection
    conn.close()
    
    #Create variables with key-value pairs of interest
    bbplayer = tableDic['bounding_boxes']
    homog = tableDic['homographies']
    bbeveryone = tableDic['bounding_boxes_bk']
    
    #Identify x,y coord for player position and create new rows with this
    
    bbplayer['posX'] = bbplayer.apply(lambda row: (row['x']+(row['width']/2)), axis =1)
    bbplayer['posY'] = bbplayer.apply(lambda row: (row['y']+ row['height']), axis =1)

    newPlay = appendNewXY(bbplayer, homog, 'posX', 'posY', 'fX','fY')
    
    return newPlay
    
  

# Locating frames with more than 2 players detected

Quick fix solution is to drop the frame with the lowest probability score.This method only works when there is 3 players in a frame. If there are frames with 3 or more detected, need to edit code.

In [7]:

def sortPlayers(dataset):
    copy = dataset.copy(deep=True)
    order = copy.sort_values(by='frame_id')
    footage = order[order['class']=='player']
    framePlayers = footage['frame_id'].value_counts()
    #frame3 stores the frame number that contains more than two players
    frame3 = framePlayers[framePlayers >2].index.values
    droppedFrames = []
    
    for i in frame3:
        issueFrame = footage.loc[footage['frame_id']==i].sort_values(by='score',ascending = False)
        issueIndex = issueFrame.index[2:]
        droppedFrames.append(issueIndex)
        footage = footage.drop(index = issueIndex)
        
#     print("Removed rows from frames: ", frame3)
#     print("Index No. of removed rows: ", droppedFrames)
    
    return footage

# Figure out Player in Far Court and Player in Near Court

dividePlayers function:

    - Takes in dataset with 2 player rows for each frame
    - sorts the dataframe by frame_id, then for each frame_id sorts by fY
    - divide dataset to two groups representing each player(odd rows even rows). Using the mean(fY) score, determines which player is near the camera, far from the camera. 
    - new column 'Court' indicates 'near' or 'far'. 

In [8]:
def dividePlayers(dataset):
    dP = dataset.sort_values(['frame_id','fY']).copy(deep=True)
    dP.loc[:,'Court'] = ''
    odd = dP.iloc[1::2,:]
    even = dP.iloc[0::2,:]
    
    
    if (odd.fY.mean() > even.fY.mean()): # Odd is Closecourt
        dP.iloc[1::2,-1] = 0
        dP.iloc[0::2,-1] = 1
    else:
        dP.iloc[1::2,-1] = 1
        dP.iloc[0::2,-1] = 0
        
    
    return dP

# Transform XY data 

Creates rotated xy coordinates in invX invY. shows what the player location would be like on the opposite side of the court.

Using invX and invY, created ssX,ssY, which are the xy coordinates for both players if they were both on the side of the court close to the camera. 

Using ssX, ssY. vertDist measures the player's longitudinal distance away from the centre net. latDist measures the lateral distance from the middle of the court. 



In [9]:
def rotate(p, origin = [914.5,1829], degrees=0):
    angle = np.deg2rad(degrees)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                 [np.sin(angle), np.cos(angle)]])
    o = np.atleast_2d(origin)
    p= np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T)

def oppXY(data):
    dataset = data.copy(deep=True)
    #playerData[newy] = playerData.apply(lambda row: total(row, homDat, xlab, ylab)[1][0], axis =1)
    dataset['invX'] = dataset.apply(lambda row: rotate(row.loc[['fX','fY']],degrees=180)[0],axis=1)
    dataset['invY'] = dataset.apply(lambda row: rotate(row.loc[['fX','fY']],degrees=180)[1],axis=1)
    dataset['ssX'] = dataset.apply(lambda row: row['invX'] if row['Court'] ==1 else row['fX'], axis= 1)
    dataset['ssY'] = dataset.apply(lambda row: row['invY'] if row['Court'] ==1 else row['fY'], axis= 1)
    dataset['vertDist'] = dataset.apply(lambda row: row['ssY'] - 1829,axis=1)
    dataset['latDist'] = dataset.apply(lambda row: row['ssX'] - 914.5,axis=1)
    dataset = dataset.drop(['x', 'y', 'width', 'height', 'mask', 'class', 'score',
       'posX', 'posY'],axis = 1)
    return dataset

In [10]:
def cleanXY(dataset):
    playerSort = sortPlayers(dataset)
    dPsort = dividePlayers(playerSort)
    transXY = oppXY(dPsort)
    
    return transXY

## Visualisation Tools

In [11]:
# Plots one rally

def plotTennis(dataset, rallyNo):
    rally = dataset[dataset['matchID']==rallyNo]
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(schema, extent=[0, 1829,3658,0])
    ax.scatter(rally.fX, rally.fY, s= 30, c = rally['frame_id'],cmap='inferno')

In [12]:
# Plot entire dataset

def plotAll(dataset):
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(schema, extent=[0, 1829,3658,0])
    ax.scatter(dataset.fX, dataset.fY, s= 30, c = dataset['frame_id'],cmap='inferno')

In [13]:
# Get one frame

def getFrame(dataset, rallyNo, frameNo):
    rally = dataset[(dataset['matchID']==rallyNo)&(dataset['frame_id']<=frameNo)]
    return rally

In [14]:
# Get one rally

def getRally(dataset, rally):
    return dataset[dataset['matchID'] == rally]

In [9]:
# plot density map of all coordinates

def plotDensity(dataset):
    xy = np.vstack([dataset['fX'], dataset['fY']])
    z = gaussian_kde(xy)(xy)
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(schemaHalf, extent=[0, 1829,3658,1830])
    ax.scatter(dataset['fX'], dataset['fY'], s= 30, c = z,cmap='inferno')
    plt.show()